In [1]:
!pip install praw

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 189.3/189.3 kB 4.1 MB/s eta 0:00:00


importing necessary libraries

In [2]:
import praw
import pandas as pd
import time
from datetime import datetime

Creating a function to create a reddit object using praw

In [3]:
def create_reddit_object():
  reddit = praw.Reddit(
      client_id = 'IJcYVVGk3-6OmX1Kk92sbA',
      client_secret = 'IwFwxj-RxswyEihBrjBijFJFyI_ndw',
      user_agent = 'python:stock_sentiment_analyzer_project:1.0 by (u/Zealousideal_Run9837)',

      check_for_async=False
  )

  return reddit


automotive stocks whose data we will scrape

In [4]:
automotive_keywords = [
    "Tesla OR TSLA",
    "Ford OR F",
    "General Motors OR GM",
    "Toyota OR TM",
    "NIO",
    "Rivian OR RIVN",
    "Lucid OR LCID",
    "Volkswagen OR VWAGY",
    "BMW OR BMWYY"
]

list of subreddit from which we will scrape the data

In [5]:
subreddits = ["stocks", "investing", "wallstreetbets", "cars", "automotive"]

creating a function to scrape following columns from each subreddit for each company.

[Company, Date/created_utc, subreddit, title, text, comments, score,num_comments,	url]

In [6]:
def scrape_reddit_data(subreddits,keywords,limit=10):
  reddit = create_reddit_object()

  scrapped_data = []

  for subreddit_name in subreddits:
    time.sleep(5)
    subreddit = reddit.subreddit(subreddit_name)
    print(f"Scraping data from r/{subreddit_name}")

    for keyword in keywords:
      print(f"Scraping data for keyword: {keyword}")
      query = f"{keyword}"
      for submission in subreddit.search(query, limit=limit):
        time.sleep(5)
        posts = {
                    "subreddit": subreddit,
                    "company": keyword.split("OR")[0].strip(),
                    "title": submission.title,
                    "text": submission.selftext,
                    "created_utc": datetime.utcfromtimestamp(submission.created_utc),
                    "score": submission.score,
                    "num_comments": submission.num_comments,
                    "comments": ", ".join([comment.body for comment in submission.comments[:10]]),
                    "url": submission.url
                }


        scrapped_data.append(posts)
        time.sleep(5)
  return scrapped_data

In [7]:
scrapped_data = scrape_reddit_data(subreddits,automotive_keywords,limit=10)

Scraping data from r/stocks
Scraping data for keyword: Tesla OR TSLA
Scraping data for keyword: Ford OR F
Scraping data for keyword: General Motors OR GM
Scraping data for keyword: Toyota OR TM
Scraping data for keyword: NIO
Scraping data for keyword: Rivian OR RIVN
Scraping data for keyword: Lucid OR LCID
Scraping data for keyword: Volkswagen OR VWAGY
Scraping data for keyword: BMW OR BMWYY
Scraping data from r/investing
Scraping data for keyword: Tesla OR TSLA
Scraping data for keyword: Ford OR F
Scraping data for keyword: General Motors OR GM
Scraping data for keyword: Toyota OR TM
Scraping data for keyword: NIO
Scraping data for keyword: Rivian OR RIVN
Scraping data for keyword: Lucid OR LCID
Scraping data for keyword: Volkswagen OR VWAGY
Scraping data for keyword: BMW OR BMWYY
Scraping data from r/wallstreetbets
Scraping data for keyword: Tesla OR TSLA
Scraping data for keyword: Ford OR F
Scraping data for keyword: General Motors OR GM
Scraping data for keyword: Toyota OR TM
Scrap

In [8]:
df = pd.DataFrame(scrapped_data)

In [9]:
df.to_csv('reddit_data.csv')

In [10]:
reddit_data = pd.read_csv('reddit_data.csv')

In [11]:
reddit_data.head()

,Unnamed: 0,subreddit,company,title,text,created_utc,score,num_comments,comments,url
0,0,stocks,Tesla,\nTesla shares drop 6% in premarket after Cybe...,Shares of electric vehicle maker Tesla tumbled...,2024-10-11 12:19:27,1699,566,"Ah, good reminder to update my timeline.\n\n~~...",https://www.reddit.com/r/stocks/comments/1g18f...
1,1,stocks,Tesla,Tesla asks shareholders to approve CEO Musk's ...,April 17 (Reuters) - Electric automaker Tesla ...,2024-04-17 11:28:13,2876,793,Firing one guy will save the shareholders $56B...,https://www.reddit.com/r/stocks/comments/1c67e...
2,2,stocks,Tesla,Tesla reports disappointing earnings for secon...,Tesla reported weaker-than-expected earnings f...,2024-07-23 20:15:03,2047,559,“It’s not an earnings company” - tsla sharehol...,https://www.reddit.com/r/stocks/comments/1eahu...
3,3,stocks,Tesla,"Tesla stock downgraded to Sell, analysts say '...",>Tesla (NASDAQ:TSLA) stock has been downgraded...,2024-07-26 11:47:00,2058,463,"What took them so long? , Only downgraded afte...",https://www.reddit.com/r/stocks/comments/1ecm2...
4,4,stocks,Tesla,Tesla earnings are out — here are the numbers,Tesla reported a 9% drop in first-quarter reve...,2024-04-23 20:08:23,1629,752,Operating margin 5.5% - now it looks more like...,https://www.reddit.com/r/stocks/comments/1cbei...


drop unwanted columns

In [12]:
reddit_data.drop(['Unnamed: 0', 'url'],axis=1,inplace=True)

Remaning created_utc as Date and setting a comman format for date

In [13]:
reddit_data = reddit_data.rename(columns={'created_utc': 'Date'})
reddit_data['Date'] = pd.to_datetime(reddit_data['Date'])

# Format to 'YYYY-MM-DD'
reddit_data['Date'] = reddit_data['Date'].dt.strftime('%Y-%m-%d')

In [14]:
reddit_data.head()

,subreddit,company,title,text,Date,score,num_comments,comments
0,stocks,Tesla,\nTesla shares drop 6% in premarket after Cybe...,Shares of electric vehicle maker Tesla tumbled...,2024-10-11,1699,566,"Ah, good reminder to update my timeline.\n\n~~..."
1,stocks,Tesla,Tesla asks shareholders to approve CEO Musk's ...,April 17 (Reuters) - Electric automaker Tesla ...,2024-04-17,2876,793,Firing one guy will save the shareholders $56B...
2,stocks,Tesla,Tesla reports disappointing earnings for secon...,Tesla reported weaker-than-expected earnings f...,2024-07-23,2047,559,“It’s not an earnings company” - tsla sharehol...
3,stocks,Tesla,"Tesla stock downgraded to Sell, analysts say '...",>Tesla (NASDAQ:TSLA) stock has been downgraded...,2024-07-26,2058,463,"What took them so long? , Only downgraded afte..."
4,stocks,Tesla,Tesla earnings are out — here are the numbers,Tesla reported a 9% drop in first-quarter reve...,2024-04-23,1629,752,Operating margin 5.5% - now it looks more like...


checking for null value

In [15]:
reddit_data.isnull().sum()

,0
subreddit,0
company,0
title,0
text,127
Date,0
score,0
num_comments,0
comments,37


- We have 37 rows with missing comments and 127 rows with missing text
- But We also notice that there are no rows with a missing title. title, text and comments all are important for analysing the sentiment
- So lets first drops rows which have both text and comments missing

In [16]:
reddit_data = reddit_data.dropna(subset=['text', 'comments'], how='all')

In [17]:
reddit_data.isnull().sum()

,0
subreddit,0
company,0
title,0
text,114
Date,0
score,0
num_comments,0
comments,24


Okay so now we are left which rows which can either have a missing text or missing comments but will have any one of the two

In [18]:
reddit_data.shape

(408, 8)


Now out of 408 rows 24 rows have missing comments. We can simply drop these rows

In [19]:
reddit_data = reddit_data.dropna(subset=['comments'])

In [20]:
reddit_data.isnull().sum()

,0
subreddit,0
company,0
title,0
text,114
Date,0
score,0
num_comments,0
comments,0


- Now if we think both title and text should have a similar type of intent and sentiment. However comments  can have diffrent sentiments and opnion.

- based on this thought we are combing both text and title column into a single column.

- this will resolve the issue of missing text column and reduce redundancy

In [21]:
reddit_data['text'] = reddit_data['title'].fillna('') + ' ' + reddit_data['text'].fillna(' ')
reddit_data['text'] = reddit_data['text'].str.strip()

<ipython-input-21-61c1d5344cfc>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  reddit_data['text'] = reddit_data['title'].fillna('') + ' ' + reddit_data['text'].fillna(' ')
<ipython-input-21-61c1d5344cfc>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  reddit_data['text'] = reddit_data['text'].str.strip()


In [22]:
reddit_data = reddit_data.drop('title',axis=1)

In [23]:
reddit_data.isnull().sum()

,0
subreddit,0
company,0
text,0
Date,0
score,0
num_comments,0
comments,0


Okay looks like reddit_data is now free of any null or missing value

## Scrapped and Cleaned Reddit Data

In [24]:
reddit_data

,subreddit,company,text,Date,score,num_comments,comments
0,stocks,Tesla,Tesla shares drop 6% in premarket after Cyberc...,2024-10-11,1699,566,"Ah, good reminder to update my timeline.\n\n~~..."
1,stocks,Tesla,Tesla asks shareholders to approve CEO Musk's ...,2024-04-17,2876,793,Firing one guy will save the shareholders $56B...
2,stocks,Tesla,Tesla reports disappointing earnings for secon...,2024-07-23,2047,559,“It’s not an earnings company” - tsla sharehol...
3,stocks,Tesla,"Tesla stock downgraded to Sell, analysts say '...",2024-07-26,2058,463,"What took them so long? , Only downgraded afte..."
4,stocks,Tesla,Tesla earnings are out — here are the numbers ...,2024-04-23,1629,752,Operating margin 5.5% - now it looks more like...
...,...,...,...,...,...,...,...
400,automotive,Rivian,Do you think the automotive industry is headin...,2023-04-09,1,2,Speaking of trends and innovations. I think we...
411,automotive,BMW,No air blowing out of vents no heat or A/C bmw...,2023-01-04,0,2,Sounds like it might have a blown fuse.
412,automotive,BMW,Thinking about a 2008 bmw 5 series 528i. Need ...,2020-05-20,2,2,"BMW E60 are beautiful cars, and we'll maintain..."
417,automotive,BMW,What If Tesla Model S Used a Carbon Fiber/ Pla...,2016-08-24,2,8,30% decrease in weight results in what increas...


- Now we need to combine this data with historial stock data (for the model) based on company and Date columns.
- Before that we need to download and format historical stock data so that we can combine the two dataset without any issue.
- I have downloaded and formatted hsitorical stock data in a seperate notebook and i am going to directly use that data by uploading the data set. You can look at the notebook by clicking on this link.

## srcapping and formatting historical data

In [25]:
import yfinance as yf
import pandas as pd

# Define the mapping of companies to tickers
ticker_map = {
    "Tesla": "TSLA",
    "Ford": "F",
    "General Motors": "GM",
    "Toyota": "TM",
    "NIO": "NIO",
    "Rivian": "RIVN",
    "Lucid": "LCID",
    "Volkswagen": "VWAGY",
    "BMW": "BMWYY"
}

# Download and save stock data for each company
for company, ticker in ticker_map.items():
    try:
        # Download data for the last 10 years
        hist_data = yf.download(ticker, period="max")
        hist_data.reset_index(inplace=True)  # Reset index to use 'Date' as a column
        hist_data['company'] = ticker  # Add a 'company' column

        # Save data to a CSV file
        hist_data.to_csv(f"{ticker}_historical_data.csv", index=False)
        print(f"Saved historical data for {company} ({ticker})")
    except Exception as e:
        print(f"Error fetching data for {company} ({ticker}): {e}")

[*********************100%***********************]  1 of 1 completed


Saved historical data for Tesla (TSLA)


[*********************100%***********************]  1 of 1 completed


Saved historical data for Ford (F)


[*********************100%***********************]  1 of 1 completed


...

Saved historical data for Lucid (LCID)


[*********************100%***********************]  1 of 1 completed


Saved historical data for Volkswagen (VWAGY)


[*********************100%***********************]  1 of 1 completed

Saved historical data for BMW (BMWYY)


In [26]:
bmw = pd.read_csv('BMWYY_historical_data.csv')
gm = pd.read_csv('GM_historical_data.csv')
ford = pd.read_csv('F_historical_data.csv')
lucid = pd.read_csv('LCID_historical_data.csv')
nio = pd.read_csv('NIO_historical_data.csv')
rivian = pd.read_csv('RIVN_historical_data.csv')
toyota = pd.read_csv('TM_historical_data.csv')
tesla = pd.read_csv('TSLA_historical_data.csv')
volkswagen = pd.read_csv('VWAGY_historical_data.csv')

In [27]:
bmw.head(3)

,Date,Adj Close,Close,High,Low,Open,Volume,company
0,NaN,BMWYY,BMWYY,BMWYY,BMWYY,BMWYY,BMWYY,NaN
1,2008-11-05 00:00:00+00:00,5.2972259521484375,9.799200057983398,9.799200057983398,9.799200057983398,9.799200057983398,217421,BMWYY
2,2008-11-06 00:00:00+00:00,4.757081031799316,8.800000190734863,9.5,8.800000190734863,8.800000190734863,24803,BMWYY


- I downloaded all the datasets seperatly and then combine them because i wanted to look at each dataset as i faced some issues while merging data earlier.
- I noticed that first row of each dataset have undesired values that might be causing the error.

- Now these rows with undesired value have no null values in date column
- so i merged all the historical dataset into one and them dropped rows with null date value.
- this resolved my issues and i was able to merge data later on smoothly

In [28]:
historical_data = pd.concat([bmw, ford, lucid, nio, rivian, toyota, tesla, volkswagen], ignore_index=True)
historical_data.dropna(subset=['Date'], inplace=True)
historical_data.isnull().sum()

,0
Date,0
Adj Close,0
Close,0
High,0
Low,0
Open,0
Volume,0
company,0


making date format of historical data same as reddit data

In [29]:
historical_data['Date'] = pd.to_datetime(historical_data['Date'])

# Format to 'YYYY-MM-DD'
historical_data['Date'] = historical_data['Date'].dt.strftime('%Y-%m-%d')

- We are using historical Data to later check our model predictions.
- So let's create a column 'Price Movement' which we will later use as our target variable.
- And then we will drop irrelevant columns

In [30]:
historical_data['Price Movement'] = historical_data.apply(
    lambda row: 'Up' if row['Close'] > row['Open'] else ('Down' if row['Close'] < row['Open'] else 'Neutral'),
    axis=1)


In [31]:
# List of columns to keep
columns_to_keep = ['Date', 'company', 'Price Movement', 'Volume']

# Drop all other columns
historical_data = historical_data.drop(
    columns=[col for col in historical_data.columns if col not in columns_to_keep])

In [32]:
historical_data.to_csv('historical_data.csv')

In [33]:
historical_data = pd.read_csv('historical_data.csv')

In [34]:
historical_data

,Unnamed: 0,Date,Volume,company,Price Movement
0,1,2008-11-05,217421,BMWYY,Neutral
1,2,2008-11-06,24803,BMWYY,Neutral
2,3,2008-11-07,32759,BMWYY,Up
3,4,2008-11-10,140799,BMWYY,Down
4,5,2008-11-11,179483,BMWYY,Neutral
...,...,...,...,...,...
41117,41125,2024-11-29,189600,VWAGY,Up
41118,41126,2024-12-02,639700,VWAGY,Down
41119,41127,2024-12-03,766600,VWAGY,Down
41120,41128,2024-12-04,813200,VWAGY,Down


- Now one think which i noticed is that i used ticker symbol of company to to download stock data from yahoo finance but in my reddit dataset i have used actual company name.

- This will cause issue in merging the two datasets.
- lets resolve this issue by using ticker format in company column of both datsets

In [35]:
# creating a ticker map
company_ticker_mapping = {
    "Tesla": "TSLA",
    "Ford": "F",
    "General Motors": "GM",
    "Toyota": "TM",
    "NIO": "NIO",
    "Rivian": "RIVN",
    "Lucid": "LCID",
    "Volkswagen": "VWAGY",
    "BMW": "BMWYY"

}

In [36]:
# renaming company names as there ticker name in reddit dataset for consistancy in both dataset
reddit_data['company'] = reddit_data['company'].map(company_ticker_mapping)

In [37]:
reddit_data.head(1)

,subreddit,company,text,Date,score,num_comments,comments
0,stocks,TSLA,Tesla shares drop 6% in premarket after Cyberc...,2024-10-11,1699,566,"Ah, good reminder to update my timeline.\n\n~~..."


In [38]:
historical_data.head(1)

,Unnamed: 0,Date,Volume,company,Price Movement
0,1,2008-11-05,217421,BMWYY,Neutral


### Looks like both my dataset have there company and date column in same format and should not cause any issue in merging both datasets

In [39]:
merged_df = pd.merge(historical_data, reddit_data, on=['company', 'Date'], how='inner')

In [40]:
merged_df

,Unnamed: 0,Date,Volume,company,Price Movement,subreddit,text,score,num_comments,comments
0,1733,2015-09-24,543502,BMWYY,Up,investing,Concern Spreads to BMW http://www.bloomberg.co...,222,149,"Well, honestly, the more car manufacturers are..."
1,1964,2016-08-24,28760,BMWYY,Down,automotive,What If Tesla Model S Used a Carbon Fiber/ Pla...,2,8,30% decrease in weight results in what increas...
2,2197,2017-07-28,37846,BMWYY,Up,stocks,What is the best way to buy shares from foreig...,1,3,You can get BMW ($BMWYY) on the American marke...
3,2512,2018-10-26,101185,BMWYY,Up,automotive,How do I fix this on my bmw?,1,1,3M headlight restoration kit\n\n
4,2606,2019-03-15,101248,BMWYY,Up,investing,Investing in BMW: BMWYY vs BMW.DE I live in US...,6,3,The DE ticker is on the German exchange traded...
...,...,...,...,...,...,...,...,...,...,...
274,41027,2024-07-12,345700,VWAGY,Up,wallstreetbets,"$RIVN up 70%, now $QS up 50% in the past 2 day...",25,32,\n**User Report**| | | |\n:--|:--|:--|:--\n**T...
275,41066,2024-09-06,652200,VWAGY,Down,wallstreetbets,Volkswagen (VWAGY) may be in financial trouble...,8,29,\n**User Report**| | | |\n:--|:--|:--|:--\n**T...
276,41102,2024-10-28,402500,VWAGY,Up,cars,Volkswagen plans to close at least three manuf...,688,270,Germany is getting hammered by the ukraine rus...
277,41104,2024-10-30,398600,VWAGY,Up,cars,[C&D] Tested: 2025 Volkswagen ID.Buzz Is the B...,352,289,234 mile range for $70K is not great when the ...


In [41]:
merged_df.isnull().sum()

,0
Unnamed: 0,0
Date,0
Volume,0
company,0
Price Movement,0
subreddit,0
text,0
score,0
num_comments,0
comments,0


In [42]:
merged_df['company'].value_counts()

,count
company,
F,40
TSLA,40
TM,36
BMWYY,35
LCID,33
VWAGY,33
NIO,31
RIVN,31


## We have Cleaned and Merged(reddit+historical) Data

- lets preprocess our data now

## Data Preprocessing

In [43]:
import nltk
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [44]:
import re
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

lemmatizer = WordNetLemmatizer()

In [45]:
def preprocess_text(text):

  text = text.lower()

  text = re.sub(r'\W', ' ', text)

  tokens = [word for word in text.split() if word not in set(stopwords.words('english'))]

  tokens = [lemmatizer.lemmatize(word) for word in tokens]

  return ' '.join(tokens)


In [46]:
merged_df['text'] = merged_df['text'].apply(preprocess_text)
merged_df['comments'] = merged_df['comments'].apply(preprocess_text)

In [47]:
merged_df

,Unnamed: 0,Date,Volume,company,Price Movement,subreddit,text,score,num_comments,comments
0,1733,2015-09-24,543502,BMWYY,Up,investing,concern spread bmw http www bloomberg com news...,222,149,well honestly car manufacturer involved practi...
1,1964,2016-08-24,28760,BMWYY,Down,automotive,tesla model used carbon fiber plastic frame si...,2,8,30 decrease weight result increase efficiency ...
2,2197,2017-07-28,37846,BMWYY,Up,stocks,best way buy share foreign company thinking pu...,1,3,get bmw bmwyy american market broker checked l...
3,2512,2018-10-26,101185,BMWYY,Up,automotive,fix bmw,1,1,3m headlight restoration kit
4,2606,2019-03-15,101248,BMWYY,Up,investing,investing bmw bmwyy v bmw de live u want buy s...,6,3,de ticker german exchange traded germanybucks ...
...,...,...,...,...,...,...,...,...,...,...
274,41027,2024-07-12,345700,VWAGY,Up,wallstreetbets,rivn 70 q 50 past 2 day rivn went since volksw...,25,32,user report total submission 1 first seen wsb ...
275,41066,2024-09-06,652200,VWAGY,Down,wallstreetbets,volkswagen vwagy may financial trouble help ri...,8,29,user report total submission 1 first seen wsb ...
276,41102,2024-10-28,402500,VWAGY,Up,cars,volkswagen plan close least three manufacturin...,688,270,germany getting hammered ukraine russia war lo...
277,41104,2024-10-30,398600,VWAGY,Up,cars,c tested 2025 volkswagen id buzz bee knee,352,289,234 mile range 70k great kia ev9 get 270 304 m...


In [48]:
merged_df.drop(['Unnamed: 0'],axis=1,inplace=True)

In [49]:
merged_df.shape

(279, 9)

In [50]:
merged_df.drop_duplicates(inplace=True)

In [51]:
merged_df.shape

(279, 9)

no duplicate values

## Now we have clean, Preprocessed and Complete data

In [52]:
merged_df.to_csv('scrapped_data.csv')

### Now use this "scrapped_data.csv" as input data in Analysis and Feature creation.ipynb Notebook